In [7]:
import pandas as pd
from pathlib import Path

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_colwidth', None)

In [3]:
DATA_DIR = "../../../data/reddit"
SUBREDDIT = "plantclinic"
SIZE = 10000
COMMENTS = Path(DATA_DIR) / f"{SUBREDDIT}_comments"
SUBMISSIONS = Path(DATA_DIR) / f"{SUBREDDIT}_submissions"
REDUCED_COMMENTS = COMMENTS.with_suffix(f'.l{SIZE}')
REDUCED_SUBMISSIONS = SUBMISSIONS.with_suffix(f'.l{SIZE}')

In [4]:
assert COMMENTS.exists()
assert SUBMISSIONS.exists()

In [5]:
! head -{SIZE} {SUBMISSIONS} > {REDUCED_SUBMISSIONS}
! head -{SIZE} {COMMENTS} > {REDUCED_COMMENTS}

In [6]:
df_comments = pd.read_json(REDUCED_COMMENTS, orient='records',lines=True)
df_submissions = pd.read_json(REDUCED_SUBMISSIONS, orient='records',lines=True)

In [63]:
df_s = df_submissions
df_s = df_s[(df_s["num_comments"] > 0) & (df_s["title"].str.len() > 100)].sort_values(by=['score'], ascending=False)
df_s = df_s[["id", "title"]]
df_s["id"] = "t3_" + df_s["id"]
df_s = df_s[df_s['title'].str.endswith("?")]
df_s

,id,title
8167,t3_bcmfhr,"A rescue mission! This guy was forgotten about last season in a local store. I’ve read they don’t mind being a little root bound, but he’s burst through his pot. Would it be best to separate him into several different pots, or replant him together in a larger pot?"
8622,t3_bf23j5,Recently bought this fiddle leaf fig! How long will it take for the leaves to start spreading out and lying more flat? Is there something I should do?
...,...,...
7601,t3_b8t52c,"My orchid seemed to be doing ok until a week ago, almost all of the flowers fell off and the stem is changing color. Does anyone know what could be going on?"
4986,t3_a860yn,What's going on with my baby? Under watering? Over? Heat stress? Or is it a normal part of its cycle?


In [64]:
link_ids = set(df_comments[df_comments["link_id"].isin(df_s["id"])]["link_id"])

In [65]:
df_c = df_comments
df_s["comment"] = ""
for l_id in link_ids:
    filtered = df_c[(df_c["link_id"] == l_id) & (df_c["parent_id"] == l_id)].sort_values(by=['score'], ascending=False)
    if filtered.shape[0] > 0:
        most_upvoted_comment = filtered.iloc[0]["body"]
        df_s.loc[df_s.id == l_id, "comment"] = most_upvoted_comment


In [66]:
df_s_final = df_s[df_s["comment"].str.len() > 0]

In [67]:
df_s_final.to_csv("test.csv")

In [55]:
df_c = df_comments
df_c = df_c[(df_c["link_id"] == "t3_bh5ioh") & (df_c["parent_id"] == "t3_bh5ioh")].sort_values(by=['score'], ascending=False)
df_c.iloc[0]["body"]

IndexError: single positional indexer is out-of-bounds

In [ ]:
df_s = df_submissions
df_s = df_s[(df_s["num_comments"] > 0) & (df_s["title"].str.len() > 10)].sort_values(by=['score'], ascending=False)
df_s = df_s[["title", "selftext"]]
pd.set_option('display.max_rows', 30)
#df_c.iloc[0]["selftext"]
# title
df_s[df_s['title'].str.endswith("?")].head(100)